Pip Install Commands

In [1]:
%pip install shapely

Note: you may need to restart the kernel to use updated packages.


Libraries

In [2]:
import os
import json
import requests
import numpy as np
import pandas as pd
import networkx as nx
from shapely.prepared import prep
from shapely.geometry import mapping, shape, Point

Const Values

In [3]:
YEAR_COLUMN_NAME = "year"
DECADE_COLUMN_NAME = "decade"
SONG_TITLE_COLUMN_NAME = "song_title"
COUNTRY_COLUMN_NAME = "country"
ARTIST_LONGITUDE_COLUMN_NAME = "artist_longitude"
ARTIST_LATITUDE_COLUMN_NAME = "artist_latitude"
ARTIST_LOCATION_COLUMN_NAME = "artist_location"
ARTIST_ID_COLUMN_NAME = "artist_id"
SONG_ID_COLUMN_NAME = "song_id"

UNKNOWN_COUNTRY_VALUE = "unknown"
MUSIC_DATA_FOLDER_PATH = "../Music Data/"

Loading Songs & Artists datasets

In [4]:
raw_songs_dataset = pd.read_csv("../Data/songs_dataset.csv")
raw_artists_dataset = pd.read_csv("../Data/artist_terms.csv")

Riaz: Merging datasets based on artist_id<br>
In the following cell i am removing the duplicate rows based on `artist_id` and only keep the first record

In [5]:
# Remove duplicates from the artist dataset based on artist_id
raw_artists_dataset = raw_artists_dataset.drop_duplicates(subset=ARTIST_ID_COLUMN_NAME, keep='first')

# Merge the datasets on artist_id
raw_music_dataset = pd.merge(raw_songs_dataset, raw_artists_dataset, on=ARTIST_ID_COLUMN_NAME, how='left')

In the above cell i merged the datasets based on artist_id and merge was on left join:
When you specify how='left', it means that all the keys from the left dataframe (in this case, the raw_songs_dataset dataframe) will be included in the merged dataframe, and only the matching keys from the right dataframe (in this case, the artist_dataset dataframe) will be added.

In other words:

All rows from the left dataframe (raw_songs_dataset) are retained.
If there are matching keys (in this case, artist_id) in the right dataframe (artist_dataset), the corresponding data from the right dataframe will be added to the merged dataframe.
If there are no matching keys in the right dataframe, the corresponding columns in the merged dataframe will be filled with NaN (missing values).

In [6]:
raw_music_dataset.head()

,song_id,song_title,year,release,tempo,loudness,duration,song_hotttnesss,artist_id,artist_name,artist_latitude,artist_longitude,artist_location,artist_hotttnesss,artist_familiarity,term
0,SOVFVAK12A8C1350D9,Tanssi vaan,1995.0,Karkuteillä,150.778,-10.555,156.55138,0.299877,ARMVN3U1187FB3A1EB,Karkkiautomaatti,NaN,NaN,NaN,0.356992,0.439604,pop rock
1,SOGTUKN12AB017F4F1,No One Could Ever,2006.0,Butter,177.768,-2.060,138.97098,0.617871,ARGEKB01187FB50750,Hudson Mohawke,55.8578,-4.24251,"Glasgow, Scotland",0.437504,0.643681,broken beat
2,SOBNYVR12A8C13558C,Si Vos Querés,2003.0,De Culo,87.433,-4.654,145.05751,NaN,ARNWYLR1187B9B2F9C,Yerba Brava,NaN,NaN,NaN,0.372349,0.448501,cumbia
3,SOHSBXH12A8C13B0DF,Tangle Of Aspens,NaN,Rene Ablaze Presents Winter Sessions,140.035,-7.806,514.29832,NaN,AREQDTE1269FB37231,Der Mystic,NaN,NaN,NaN,0.000000,0.000000,hard trance
4,SOZVAPQ12A8C13B63C,"Symphony No. 1 G minor ""Sinfonie Serieuse""/All...",NaN,Berwald: Symphonies Nos. 1/2/3/4,90.689,-21.420,816.53506,NaN,AR2NS5Y1187FB5879D,David Montgomery,NaN,NaN,NaN,0.109626,0.361287,ragtime


In [7]:
raw_music_dataset.isna().sum()

song_id                    0
song_title                 2
year                  484270
release                    7
tempo                      0
loudness                   0
duration                   0
song_hotttnesss       417782
artist_id                  0
artist_name                0
artist_latitude       641766
artist_longitude      641766
artist_location       487546
artist_hotttnesss         12
artist_familiarity       185
term                    3767
dtype: int64

In [8]:
raw_music_dataset.isna().sum().sum()

2677103

Shartil: For now I am going to delete all rows with missing data.

In [9]:
music_dataset = raw_music_dataset.dropna()

In [10]:
len(music_dataset)

126903

Shartil: Adding decade column to dataset

In [11]:
music_dataset = music_dataset.assign(decade=lambda row: (row[YEAR_COLUMN_NAME].astype(int) // 10) * 10)

In [12]:
min_decade = music_dataset[DECADE_COLUMN_NAME].min()
max_decade = music_dataset[DECADE_COLUMN_NAME].max()

decade_array = np.linspace(min_decade, max_decade, 10, dtype=int)

Shartil: Evenly selecing 1000 songs, and saving them as the final dataframe

In [13]:
print(music_dataset.shape)

music_dataset = music_dataset.iloc[::127] # returns dataframe with (1000, 13)

print(music_dataset.shape)

(126903, 17)
(1000, 17)


Najeeb: Introducing a new column "country" based on Latitude and Longitude.

This code is used to load geographical data from a Local GeoJSON file, process it, and subsequently determine which country a given set of Latitudes and Logitudes coordinates falls into. The country names extracted from GeoJSON file are then inserted into a new column in a dataset.

- The "json.load" function reads the file and convert it into a Python dictionary ('geojson_data').

- An Empty dictionary named 'countries' is initiated to store the coordinated data associated with each country.

- The script iterates over each feature in the 'features' array of the 'geojson_data'. Each features represents a country.

- For each feature, the geometry ('geom') and the administrative name of the country is extracted.

- The geometry is then processed with a function 'prep' applied to 'shape(geom)'. This likely involves creating a geometric shape from the geometry data and preparing it for fast spatial queries. The processed geometry is dtored in the 'countries' dictionary with the country name as key.

- A function 'get_country' is defined which takes longitudes ('lon') and latitude ('lat') as arguments and creates a ('Point') object from coordinates.

- It then iterates  over the 'countries' dictionary and check whether the point is contained within any of the country geometrics using the 'contains' method of the geometry.

- if a containing country is found, the function returns the country's name. if no containing country is found, it returns a value 'UNKNOWN_COUNTRY_VALUE'

- A new column in the dataset ('msic_dataset') is populated by applying 'get_country' function to each row. In this way the country column is added to music_dataset based on latitude and logitude columns.


In [14]:
# Fetch and process the geojson data from a local file
with open(r'..\Data\countries.geojson.json', 'r') as file:
    geojson_data = json.load(file)

countries = {}
for feature in geojson_data["features"]:
    geom = feature["geometry"]
    country = feature["properties"]["ADMIN"]
    countries[country] = prep(shape(geom))

# Function to get country name from latitude and longitude
def get_country(lon, lat):
    point = Point(lon, lat)
    for country, geom in countries.items():
        if geom.contains(point):
            return country

    return UNKNOWN_COUNTRY_VALUE

# Apply the function to create a new 'country' column
music_dataset[COUNTRY_COLUMN_NAME] = music_dataset.apply(
    lambda row: get_country(row[ARTIST_LONGITUDE_COLUMN_NAME], 
    row[ARTIST_LATITUDE_COLUMN_NAME]), 
    axis=1
    )

Shartil: Deleting redundant columns 

In [15]:
music_dataset = music_dataset.drop(
    [
        ARTIST_LATITUDE_COLUMN_NAME,
        ARTIST_LONGITUDE_COLUMN_NAME,
        ARTIST_LOCATION_COLUMN_NAME,
        SONG_ID_COLUMN_NAME,
        ARTIST_ID_COLUMN_NAME
    ], 
    axis=1)

Shartil: Deleting all rows with "unknown" as the country value

In [16]:
music_dataset = music_dataset[music_dataset[COUNTRY_COLUMN_NAME] != UNKNOWN_COUNTRY_VALUE]

In [17]:
music_dataset.reset_index(drop=True, inplace=True)
music_dataset.head()

,song_title,year,release,tempo,loudness,duration,song_hotttnesss,artist_name,artist_hotttnesss,artist_familiarity,term,decade,country
0,No One Could Ever,2006.0,Butter,177.768,-2.060,138.97098,0.617871,Hudson Mohawke,0.437504,0.643681,broken beat,2000,United Kingdom
1,Medicine,2002.0,Closed.,131.246,-8.216,215.14404,0.505487,Drag The River,0.417750,0.612363,alternative country,2000,United States of America
2,Wild Thing,1966.0,All The Hits Plus More By The Troggs,104.824,-13.375,162.53342,0.424091,The Troggs,0.456893,0.688242,british invasion,1960,United Kingdom
3,Walk In The Fire,1989.0,Walk In The Fire,92.182,-4.253,213.99465,0.000000,Strangeways,0.401048,0.553699,tech house,1980,United States of America
4,Barbara Allen,1973.0,Angel Clare,65.116,-13.123,322.19383,0.586669,Art Garfunkel,0.410432,0.663882,soft rock,1970,United States of America


In [18]:
if not os.path.isdir(MUSIC_DATA_FOLDER_PATH):
    os.mkdir(MUSIC_DATA_FOLDER_PATH)

music_dataset.to_csv(f"{MUSIC_DATA_FOLDER_PATH}/music_dataset.csv", mode='w+')

Shartil: Now I am going to create the graph

In [19]:
music_graph = nx.Graph()

In [20]:
music_graph.add_nodes_from(decade_array.tolist())
music_graph.add_nodes_from(music_dataset[COUNTRY_COLUMN_NAME].unique().tolist())
music_graph.add_nodes_from(music_dataset[SONG_TITLE_COLUMN_NAME].tolist())

In [21]:
relationships = []
for index, row in music_dataset.iterrows():
    current_song_title = row[SONG_TITLE_COLUMN_NAME]
    current_decade = row[DECADE_COLUMN_NAME]
    current_country = row[COUNTRY_COLUMN_NAME]

    relationships.append((current_decade, current_song_title, {"label": "release_decade"}))
    relationships.append((current_country, current_song_title, {"label": "release_country"}))

music_graph.add_edges_from(relationships)

In [22]:
print(music_graph)

Graph with 1030 nodes and 1960 edges


In [23]:
def get_songs_by_criteria(music_graph, given_criteria):
    selected_songs = [ song for song in music_graph[given_criteria].keys()]
    return selected_songs

In [24]:
def print_length_and_items(given_list, amont_of_items = 50):
    print(len(given_list))
    print(given_list[:amont_of_items])

In [25]:
decade_input = 1990

decade_songs = get_songs_by_criteria(music_graph, decade_input)
print_length_and_items(decade_songs)

247
['The Road', 'Cradle', 'Johnny Come Lately', 'Last Train Home', 'Warsong', 'Stranger In Paradise', 'Pain_ Death_ Hate', 'No Wheels', 'Whatchugot (Album Version)', 'The Angel Of The Forever Sleep', 'Oh!', 'Bubble Brain', 'Going Home', 'Can Blue Men Sing The Whites', 'Breaking Up', 'By The Sea (Album Version)', 'Blue', 'Detention', 'Over', 'I Hear A Rhapsody', 'Women Respond To Bass (Vocal)', 'A Famous Rabbit', "Fastest Healin' Broken Heart", 'Dedication', 'Down in Spirit', 'Justice Tonight/Kick It Over', 'Peace', 'The Nerve', 'Get Up Little Suzie', 'Visionz', 'Thank You', 'Stop The Wheel', 'Ai Naviganti In Ascolto', 'Here Comes (Album)', 'DND', 'Taken', 'Chalawa', 'A Head With Wings (Album Version)', 'I Still Love You', 'Tomorrow (Live)', 'Through The Night', 'In My House', '1', 'Raindrops', 'Misunderstood', 'Symphony of the dead', 'Rakimou', 'Verge Of A Miracle', 'Zion Youth', 'Endorphin']


In [26]:
country_input = "Sweden"

country_songs = get_songs_by_criteria(music_graph, country_input)
print_length_and_items(country_songs)

16
['Pain_ Death_ Hate', 'Make Me Beautiful', 'When Kingdoms fall', 'Symphony of the dead', 'The second time around', 'Clearly Insane', 'Sov lilla pappa', 'Through Gardens Of Grief', 'Baby!', 'One Step Up', "Inside what's within behind", 'Leg 3', 'Begging For Mercy', 'Miracle (Bogdan Irkük Remix)', 'Legions Of The Black Light', 'Black Hearted Son']


Shartil: Now let's get the intersection of the lists<br>
This code was taken from this [StackOverflow answer](https://stackoverflow.com/a/3697438/9609586)

In [27]:
print("The songs from Sweden that were released in 1990:")

result_list = list(set(decade_songs) & set(country_songs))
print_length_and_items(result_list)

The songs from Sweden that were released in 1990:
4
['Through Gardens Of Grief', 'Pain_ Death_ Hate', "Inside what's within behind", 'Symphony of the dead']
